### 데이터 로드

In [1]:
from langchain.document_loaders import PyPDFLoader

# 약관문서 샘플 : https://www.kbinsure.co.kr/CG802030002.ec
# 20240401 일자 버전.
# PDF 파일 로드
loader = PyPDFLoader("/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf")
document = loader.load()
document[0].page_content[:200] # 내용 추출

'KB개인상해보험'

### 데이터 분할

In [2]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

print("document:", len(document))
print("texts:", len(texts))

document: 203
texts: 203


### 저장 및 검색

In [3]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# OpenAI 임베딩 사용
import os
os.environ['OPENAI_API_KEY'] = open('API_KEY', 'r').read()

# 임베딩
embeddings = OpenAIEmbeddings()

# 저장할 경로 지정
DB_PATH = "./chroma_db"

# Chroma DB 에 저장
docsearch = Chroma.from_documents(texts, embeddings, persist_directory=DB_PATH, collection_name="my_db")

# retriever 가져옴
retriever = docsearch.as_retriever()

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5859/183684450.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [4]:
retriever.get_relevant_documents("보험금 지급사유")

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5859/686092555.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("보험금 지급사유")


[Document(metadata={'page': 15, 'source': '/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf'}, page_content='- 5 -\n⑥ 계약자, 피보험자 또는 보험수익자는 제15조(알릴 의무 위반의 효과) 및 제2항의 보험금 지급사유조사와 관련하여 의료기관, 국민건강보험공단, 경찰서 등 관공서에 대한 회사의 서면에 의한 조사요청에 동의하여야 합니다. 다만, 정당한 사유 없이 이에 동의하지 않을 경우 사실 확인이 끝날 때까지 회사는 보험금 지급지연에 따른 이자를 지급하지 않습니다.⑦ 회사는 제6항의 서면조사에 대한 동의 요청시 조사목적, 사용처 등을 명시하고 설명합니다.제9조(보험금 받는 방법의 변경) ① 계약자(보험금 지급사유 발생 후에는 보험수익자)는 회사의 사업방법서에서 정한 바에 따라 보험금의 전부 또는 일부에 대하여 나누어 지급받거나 일시에 지급받는 방법으로 변경할 수 있습니다.② 회사는 제1항에 따라 일시에 지급할 금액을 나누어 지급하는 경우에는 나중에 지급할 금액에 대하여 ‘보험개발원이 공시하는 월평균 정기예금이율’을 연단위 복리로 계산한 금액을 더하며, 나누어 지급할 금액을 일시에 지급하는 경우에는 ‘보험개발원이 공시하는 월평균 정기예금이율’을 연단위 복리로 할인한 금액을 지급합니다.제10조(주소변경통지) ① 계약자(보험수익자가 계약자와 다른 경우 보험수익자를 포함합니다)는 주소 또는 연락처가 변경된 경우에는 지체없이 그 변경내용을 회사에 알려야 합니다.② 제1항에서 정한대로 계약자 또는 보험수익자가 변경내용을 알리지 않은 경우에는 계약자 또는 보험수익자가 회사에 알린 최종의 주소 또는 연락처로 등기우편 등 우편물에 대한 기록이 남는 방법으로 회사가 알린 사항은 일반적으로 도달에 필요한 기간이 지난 때에 계약자 또는 보험수익자에게 도달된 것으로 봅니다.제11조(보험수익자의 지정) 보험수익자를 지정하지 않은 때에는 보험수익자를 제3조(보험금

### 프롬프트 템플릿

In [5]:
# langchain hub 에서 Prompt 다운로드 예시
# https://smith.langchain.com/hub/rlm/rag-prompt

from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

/Users/jaesolshin/Documents/GitHub/pgvector-python/venv/lib/python3.12/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


### 제네레이터

In [6]:
# LLM
from langchain.chat_models import ChatOpenAI

# ChatGPT 모델 지정
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5859/670865594.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


In [7]:
# RAG chain 생성
from langchain.schema.runnable import RunnablePassthrough

# pipe operator를 활용한 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

### 테스트

In [8]:
rag_chain.invoke("보험금의 지급사유를 알려 줘").content

'보험금의 지급사유는 보험계약에서 정한 특정 조건이 충족될 때 발생합니다. 예를 들어, 피보험자가 사고로 인해 손해를 입었을 때 보험금 지급이 이루어질 수 있습니다. 구체적인 지급사유는 보험계약의 약관에 명시되어 있습니다.'

In [9]:
rag_chain.invoke("오늘 점심 메뉴는?").content

"I don't know."